In [37]:
import pandas as pd
import json
from help_old.matcher_query.main_matcher import result_for_match
import psycopg2
from tqdm import tqdm
from help_old.aux_function import *
import numpy as np
import pandas as pd

In [8]:
df1=pd.read_csv("./prepare_data_based_on_gold_standard/Final_prepared_data_for_matching.csv")

In [10]:
input_data=df1

In [11]:
input_data=input_data[input_data["ref_seg_dic"]!="not well-formed (invalid token): line 1, column 974"]

In [13]:
#!export PATH=~/anaconda2/bin:$PATH
#!conda install bibtexparser --channel conda-forge

# =======================EXAMPLE===============================

In [14]:
#example
temp_val=eval(input_data["ref_seg_dic"][2])

In [15]:
input_data.iloc[2]

new_id                                                 f1_646040
ref_id                                                    646040
sowi_id                                 ['gesis-solis-00244164']
sourcefile                                                 12608
ref_text_x     Heins, Volker/Flitner, Michael: Biologische Re...
ref_text_y     Heins, Volker/Flitner, Michael: Biologische Re...
ref_seg        <author><surname prob="0.9938">Heins</surname>...
ref_seg_dic    {"author": [[{"score": "0.9938", "surname": "H...
Name: 2, dtype: object

In [16]:
dict_generator(temp_val)

{'ID': 'id_autogenrated',
 'author': 'Heins',
 'journal': 'Flitner et al',
 'pages': '13,38',
 'title': 'Biologische Ressourcen und „Life Politics',
 'year': ['1998']}

In [17]:
type(dict_generator(temp_val)["number"])

KeyError: 'number'

In [20]:
res=result_for_match(dict_generator(temp_val))

In [21]:
df=pd.DataFrame(res)

In [22]:
df[df.match_id!="not_match"]

,keys_flag,match_id
14,"title,year,author",[{0: {'person_author_normalized_str_mv': ['hei...
19,"title,year",[{0: {'person_author_normalized_str_mv': ['hei...
20,"title,author",[{0: {'person_author_normalized_str_mv': ['hei...
22,title,[{0: {'person_author_normalized_str_mv': ['hei...


# ====================create basic table=====================

In [23]:
conn = psycopg2.connect("dbname='dbexcitetest' user='postgres' host='localhost' password='sql!123'")

In [24]:
cur = conn.cursor()

# import in basic table

In [25]:
for index,item in input_data[["new_id", "sourcefile", "ref_text_x", "ref_seg_dic"]].iterrows():
    query = """INSERT INTO BNA_source_table_match_test(ref_id, sourcefile , ref_text , segcit_dict) VALUES (%s, %s, %s, %s)"""
    data = (item["new_id"], item["sourcefile"], item["ref_text_x"],item["ref_seg_dic"])
    cur.execute(query, data)
conn.commit()

# Read from this table

In [25]:
query = """select * from BNA_source_table_match_test"""
cur.execute(query)
rows = cur.fetchall()

In [26]:
dfselect_row=pd.DataFrame(rows)

In [27]:
dfselect_row.head(2)

,0,1,2,3
0,f1_831418,10037,"Bosch, G.: Qualifizieren statt entlassen - Bes...","{""author"": [[{""score"": ""0.9656"", ""surname"": ""B..."
1,f1_1064777,31981,"Stahlmann, Günther (2006): 1 - Euro - Jobs zwi...","{""author"": [[{""score"": ""0.9931"", ""surname"": ""S..."


# Try fill dict table and result_sowiport

In [ ]:
cur = conn.cursor()
for index,item in tqdm(dfselect_row.iterrows()):
    try:
        query = """INSERT INTO BNA_dict_table_match_test(ref_id, Dict_for_matching) VALUES (%s, %s)"""
        data = (item[0], json.dumps(dict_generator(json.loads(item[3]))))
        cur.execute(query, data)
        sowi_result_records=result_for_match(dict_generator(json.loads(item[3])))
        for item1 in sowi_result_records:
            query = """INSERT INTO BNA_records_table_sowimatch_test(ref_id, resultfromsowi) VALUES (%s, %s)"""
            data = (item[0], json.dumps(item1))
            cur.execute(query, data)  
    except Exception as e: 
        print(e)
        print(index)
        print(item)
        print("===")
conn.commit()

In [28]:
from multiprocessing import Pool
from tqdm import tqdm_notebook

def f(x):
    ls=[]
    try:
        #query = """INSERT INTO BNA_dict_table_match_test(ref_id, Dict_for_matching) VALUES (%s, %s)"""
        data1 = (x[1][0], json.dumps(dict_generator(json.loads(x[1][3]))))
        sowi_result_records=result_for_match(dict_generator(json.loads(x[1][3])))
        for item1 in sowi_result_records:
            #query = """INSERT INTO BNA_records_table_sowimatch_test(ref_id, resultfromsowi) VALUES (%s, %s)"""
            data = (x[1][0], json.dumps(item1))
            ls.append(data)
        return [data1,ls]
            
    except Exception as e: 
        print(e)
        print(index)
        print(x[1])
        print("===")
    
def f2():
    p = Pool(2)
    #return list(tqdm(p.imap(f, dfselect_row.iterrows())))
    return list(tqdm_notebook(p.imap(f, dfselect_row.iterrows())))
    

In [29]:
listofresult=f2()

In [30]:
listofresult[:1]

[[('f1_831418',
   '{"ID": "id_autogenrated", "year": ["1990"], "author": "Bosch"}'),
  []]]

In [66]:
conn = psycopg2.connect("dbname='dbexcitetest' user='postgres' host='localhost' password='sql!123'")
cur = conn.cursor()
for item in tqdm_notebook(listofresult):
    try:
        query = """INSERT INTO BNA_dict_table_match_test(ref_id, Dict_for_matching) VALUES (%s, %s)"""
        cur.execute(query, item[0])
    except:
        print(item[0])
        print("error")
        break

In [67]:
conn.commit()

1- ref_id, 1-1- keys_flag, 1-2- resultfromsowi, 2- sowi_id, 3- sowi_title_full, 4- sowi_title_sub, 5- sowi_facet_person_str_mv,
6- sowi_person_author_normalized_str_mv, 7- sowi_norm_title_str, 8- sowi_norm_title_full_str, 9- soiw_norm_publishDate_str,
10- sowi_norm_pagerange_str, 11- sowi_recorddoi_str_mv, 12- sowi_norm_volume_str, 13- sowi_norm_issue_str,
14- sowi_journal_title_txt_mv, 15- sowi_journal_short_txt_mv, 16-position_in_list

In [52]:
def dicttotuplesowiport_psql(ref_id,keys_flag,item3):
            positioninlist=list(item3.keys())[0]
            item3=item3[positioninlist]
            return (ref_id,keys_flag,json.dumps(item3),item3.get("id","nv"),
            item3.get("title_full","nv"),item3.get("title_sub","nv"),item3.get("facet_person_str_mv","nv"),
            item3.get("person_author_normalized_str_mv","nv"),item3.get("norm_title_str","nv"),
            item3.get("norm_title_full_str","nv"),item3.get("norm_publishDate_str","nv"),
            item3.get("norm_pagerange_str","nv"),item3.get("recorddoi_str_mv","nv"),
            item3.get("norm_volume_str","nv"),item3.get("norm_issue_str","nv"),item3.get("journal_title_txt_mv","nv"),
            item3.get("journal_short_txt_mv","nv"),positioninlist)

In [59]:
cur = conn.cursor()
query = """INSERT INTO bna_records_table_sowimatch_test
        (ref_id,keys_flag,resultfromsowi,sowi_id,
        sowi_title_full,sowi_title_sub,sowi_facet_person_str_mv,sowi_person_author_normalized_str_mv,
        sowi_norm_title_str,sowi_norm_title_full_str,soiw_norm_publishdate_str,sowi_norm_pagerange_str,
        sowi_recorddoi_str_mv,sowi_norm_volume_str,sowi_norm_issue_str,sowi_journal_title_txt_mv,
        sowi_journal_short_txt_mv,position_in_list) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                                                            %s, %s, %s, %s, %s, %s)"""

for item in listofresult:
    for item1 in item[1]:
        item11=json.loads(item1[1])
        mat_records=item11["match_id"]
        if str(mat_records)!="not_match":
            if len(mat_records)>0:
                for item3 in mat_records:
                    cur.execute(query,dicttotuplesowiport_psql(item1[0],item11["keys_flag"],item3))

In [60]:
conn.commit() 

# ========remove_dublicate_sowi_id===========

In [99]:
conn = psycopg2.connect("dbname='dbexcitetest' user='postgres' host='localhost' password='sql!123'")
cur = conn.cursor()
query = """select ref_id, sowi_id ,min(position_in_list), min(auxnr), count(*) from bna_records_table_sowimatch_test group by ref_id, sowi_id;"""
cur.execute(query)
rows = cur.fetchall()

In [100]:
df67=pd.DataFrame(rows)

In [102]:
df67.head(5)

,0,1,2,3,4
0,f1_1083556,gesis-solis-00151722,0,944,8
1,f1_933880,csa-assia-68497,0,2851,8
2,f2_705,csa-assia-70854,4,4320,2
3,f2_163,csa-pais-1987-0600511,0,4477,2
4,f1_916667,gesis-solis-00242984,1,2608,18


In [109]:
cur = conn.cursor()
ls_delete_item=[]
for index,item in df67.iterrows():
    query = """select min(auxnr) FROM bna_records_table_sowimatch_test WHERE ref_id = %s AND sowi_id = %s AND position_in_list = %s;"""
    cur.execute(query,(str(item[0]),str(item[1]),str(item[2],)))
    conn.commit()
    rows = cur.fetchall()
    ls_delete_item.append((str(item[0]),str(item[1]),rows[0][0]))

In [110]:
for item in ls_delete_item:
    query = """Delete FROM bna_records_table_sowimatch_test WHERE ref_id = %s AND sowi_id = %s AND auxnr <> %s;"""
    cur.execute(query,(str(item[0]),str(item[1]),str(item[2],)))
    conn.commit()

# ===============apply json loads==============

In [23]:
input_data["ref_seg_dic"]=input_data["ref_seg_dic"].apply(json.loads)

/home/administrator/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
input_data["Maped_dict"]=input_data["ref_seg_dic"].apply(dict_generator)

# ======================apply matching=======================

In [45]:
tqdm.pandas()

In [46]:
input_data["result"]=input_data["Maped_dict"].progress_apply(result_for_match)

 12%|█▏        | 95/816 [02:21<11:56,  1.01it/s]

KeyboardInterrupt: 

In [47]:
input_data["result"].iloc[1]

KeyError: 'result'

# =======================================================

In [14]:
lsma=[]
for item in list(df[df.match_id!="not_match"]["match_id"]):
    for item1 in item:
        lsma.append(list(item1.values())[0])

In [15]:
set(pd.DataFrame(lsma)["id"])

{'dzi-solit-0159225', 'gesis-solis-00360757', 'iab-litdok-K060329801'}

# ==============================================================

In [16]:
from help_old.matcher_query.listofcombination_precision import *

In [17]:
main_list_prescion_keycombi(0.6)

/home/notebook/Last_match_version_try/help_old/matcher_query/listofcombination_precision.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  wholematch1["avg_precision"] > precision_thershold].sort_values(["avg_precision", "avg_tp", "avg_recall"],


# =============================================

In [14]:
input_data["ref_seg_dic"][:2].apply(lambda x: print(x + "\n"+ "==========="))

{"author": [[{"score": "0.9656", "surname": "Bosch"}, {"given-names": "G", "score": "0.9714"}]], "other": [{"score": "0.7765", "value": "Opladen"}], "year": [{"score": "0.9555", "value": "1990"}]}
{"author": [[{"score": "0.9931", "surname": "Stahlmann"}, {"given-names": "Günther", "score": "0.9815"}]], "year": [{"score": "0.9999", "value": "2006"}], "title": [{"score": "0.6995", "value": "1"}, {"score": "0.7465", "value": "-"}, {"score": "0.8167", "value": "Euro"}, {"score": "0.8145", "value": "-"}, {"score": "0.9109", "value": "Jobs"}, {"score": "0.9201", "value": "zwischen"}, {"score": "0.9235", "value": "Arbeitsvermittlung"}, {"score": "0.9210", "value": "und"}, {"score": "0.9185", "value": "Abstellgleis"}], "other": [{"score": "0.6871", "value": "In"}], "source": [{"score": "0.9893", "value": "Zeitschrift"}, {"score": "0.9877", "value": "für"}, {"score": "0.9856", "value": "Sozialhilfe"}, {"score": "0.9319", "value": "und"}, {"score": "0.8745", "value": "Sozialgesetzbuch"}], "pag

0    None
1    None
Name: ref_seg_dic, dtype: object